### Libraries

In [3]:
import pandas as pd
import numpy as np 

import warnings
import os

from unidecode import unidecode

import wbgapi as wb

In [5]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
warnings.filterwarnings("ignore")
os.chdir("C:\\Users\\joaos\\Documents\\GitHub\\pred_se_ml\data")

### Shadow economy data from Medina, L., & Schneider, F. (2017)

In [6]:
# Importing shadow economy data from Medina, L., & Schneider, F. (2017)
se_medina = pd.read_excel("shadow_economy_medina.xlsx")
se_medina.head()

,Country,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Albania,43.18,40.18,39.45,40.07,39.18,37.07,37.59,38.16,36.04,35.3,36.04,33.67,32.64,31.72,30.89,29.58,28.53,27.12,26.91,26.10,25.41,25.52,25.68,25.78,26.21
1,Algeria,34.96,36.14,38.16,38.88,37.64,37.68,37.18,38.28,36.11,34.2,33.58,31.90,29.62,27.76,24.93,24.44,24.21,24.07,25.90,25.89,27.37,26.94,25.98,25.74,23.98
2,Angola,50.17,47.80,55.43,50.48,52.47,46.30,50.48,49.21,48.64,48.8,46.14,48.40,48.86,46.81,43.84,41.23,37.13,35.26,36.25,36.54,36.49,36.60,35.92,34.53,35.25
3,Argentina,25.22,24.41,26.59,26.22,27.18,25.32,25.20,24.00,25.83,25.4,26.94,26.19,25.37,24.32,23.21,22.63,21.93,21.87,22.97,21.64,20.80,21.62,21.57,22.02,24.99
4,Armenia,46.65,49.50,48.63,44.66,47.14,47.48,46.41,45.81,46.85,46.6,47.61,44.11,42.08,43.57,41.03,41.38,39.47,35.39,41.04,40.14,38.46,35.52,34.56,34.78,35.96


In [7]:
# Columns' names
se_medina.columns

Index(['Country ',       1991,       1992,       1993,       1994,       1995,
             1996,       1997,       1998,       1999,       2000,       2001,
             2002,       2003,       2004,       2005,       2006,       2007,
             2008,       2009,       2010,       2011,       2012,       2013,
             2014,       2015],
      dtype='object')

In [8]:
# Removing the blank space and renaming the Country column
se_medina = se_medina.rename(columns={'Country ': 'country'})

# Reshaping from wide format to long format
se_medina = se_medina.set_index(se_medina['country']).stack().reset_index(name='se_medina').rename(columns={'level_2': 'year'})
se_medina.head()

,country,level_1,se_medina
0,Albania,country,Albania
1,Albania,1991,43.18
2,Albania,1992,40.18
3,Albania,1993,39.45
4,Albania,1994,40.07


In [9]:
# Renaming the column 'level_1' to 'year'
se_medina = se_medina.rename(columns={'level_1':'year'})

# Converting the 'se_medina' column to numeric
se_medina["se_medina"] = pd.to_numeric(se_medina.se_medina, errors = 'coerce')

# Droping 'na' observations
se_medina = se_medina.dropna()

Due to different data sources, the countries names may vary from one source to another, bellow some countries' names going to be renamed for future data joining

In [10]:
se_medina['country'] = [x.replace('CentralAfricanRepublic', 'central_african_republic') for x in se_medina['country']]
se_medina['country'] = [x.replace('Congo, Dem, Rep,', 'congo_dem_rep') for x in se_medina['country']]
se_medina['country'] = [x.replace('GuineaBissau', 'guinea_bissau') for x in se_medina['country']]
se_medina['country'] = [x.replace('Hong Kong SAR, China', 'hong_kong') for x in se_medina['country']]
se_medina['country'] = [x.replace('Syrian Arab, Rep,', 'Syria') for x in se_medina['country']]
se_medina['country'] = [x.replace('Brunei Darussalam', 'brunei') for x in se_medina['country']]
se_medina['country'] = [x.replace('swaziland', 'eswatini') for x in se_medina['country']]

In [11]:
# Selecting only the first name of each country
se_medina['country'] = se_medina['country'].str.split(',').str[0].str.strip()

# Only lowercase letters
se_medina['country'] = se_medina['country'].str.lower()

# Removing accentuation
se_medina['country'] = se_medina['country'].apply(lambda x: unidecode(x))

# replacing whitespace with _ in country
se_medina['country'] = [x.replace(' ', '_') for x in se_medina['country']] 

### Collecting the dependent variables from the World Bank API

In [12]:
# List of the countries 
countries = ['ALB', 'DZA', 'AGO', 'ARG', 'ARM', 'AUS', 'AUT', 'AZE', 'BHS', 'BHR', 'BGD', 'BLR', 'BEL', 'BLZ', 'BEN', 'BTN', 'BOL', 'BIH', 'BWA', 'BRA', 'BRN',
            'BGR', 'BFA', 'BDI', 'CPV', 'KHM', 'CMR', 'CAN', 'CAF', 'TCD', 'CHL', 'CHN', 'COL', 'COM', 'COD', 'COG', 'CRI', 'CIV', 'HRV', 'CYP', 'CZE', 'DNK',
            'DOM', 'ECU', 'EGY', 'SLV', 'GNQ', 'ERI', 'EST', 'ETH', 'FJI', 'FIN', 'FRA', 'GAB', 'GMB', 'GEO', 'DEU', 'GHA', 'GRC', 'GTM', 'GIN', 'GNB', 'GUY',
            'HTI', 'HND', 'HKG', 'HUN', 'ISL', 'IND', 'IDN', 'IRN', 'IRL', 'ISR', 'ITA', 'JAM', 'JPN', 'JOR', 'KAZ', 'KEN', 'KOR', 'KWT', 'KGZ', 'LAO', 'LVA',
            'LBN', 'LSO', 'LBR', 'LBY', 'LTU', 'LUX', 'MDG', 'MWI', 'MYS', 'MDV', 'MLI', 'MLT', 'MRT', 'MUS', 'MEX', 'MDA', 'MNG', 'MAR', 'MOZ', 'MMR', 'NAM',
            'NPL', 'NLD', 'NZL', 'NIC', 'NER', 'NGA', 'NOR', 'OMN', 'PAK', 'PNG', 'PRY', 'PER', 'PHL', 'POL', 'PRT', 'QAT', 'ROU', 'RUS', 'RWA', 'SAU', 'SEN',
            'SLE', 'SGP', 'SVK', 'SVN', 'SLB', 'ZAF', 'ESP', 'LKA', 'SUR', 'SWZ', 'SWE', 'CHE', 'SYR', 'TJK', 'TZA', 'THA', 'TGO', 'TTO', 'TUN', 'TUR',
            'UGA', 'UKR', 'ARE', 'GBR', 'USA', 'URY', 'VEN', 'VNM', 'YEM', 'ZMB', 'ZWE']

In [13]:
# List of the variables
series = ['NY.GDP.DEFL.KD.ZG', # Inflation, GDP deflator (annual %)
          'SL.UEM.TOTL.NE.ZS', # Unemployment, total (% of total labor force) (national estimate)
          'NE.TRD.GNFS.ZS', # Trade (% of GDP)
          'BX.KLT.DINV.WD.GD.ZS', # Foreign direct investment, net inflows (% of GDP)
          'NE.CON.GOVT.ZS', # General government final consumption expenditure (% of GDP)
          'IC.REG.PROC', # Start-up procedures to register a business (number)
          'IC.REG.COST.PC.ZS', # Cost of business start-up procedures (% of GNI per capita)
          'IC.REG.DURS', # Time required to start a business (days)
          'IC.PRP.DURS', # Time required to register property (days)
          'IC.TAX.DURS', # Time to prepare and pay taxes (hours)
          'NY.GDP.PCAP.CD' # GDP per capita (current US$)
         ]

In [14]:
# Collecting the data
wb_variables = wb.data.DataFrame(series=series, 
                       economy=countries, 
                       time=range(1991, 2016, 1), 
                       labels=False).reset_index()

In [15]:
wb_variables.head()

,economy,series,YR1991,YR1992,YR1993,YR1994,YR1995,YR1996,YR1997,YR1998,YR1999,YR2000,YR2001,YR2002,YR2003,YR2004,YR2005,YR2006,YR2007,YR2008,YR2009,YR2010,YR2011,YR2012,YR2013,YR2014,YR2015
0,AGO,BX.KLT.DINV.WD.GD.ZS,6.388693,3.466081,4.964921,3.837037,8.529489,2.76277,5.36336,17.121191,40.167251,9.623866,24.009075,11.406192,20.081014,9.329239,-3.526657,-0.072001,-1.368762,1.896314,3.136661,-3.851112,-2.704873,-1.143768,-5.380131,2.690006,11.081339
1,AGO,IC.PRP.DURS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,335.000000,335.000000,335.000000,335.000000,335.000000,190.000000,190.000000,190.000000,190.000000,190.000000,190.000000,190.000000
2,AGO,IC.REG.COST.PC.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1316.400000,910.000000,653.800000,498.200000,343.700000,196.800000,151.100000,226.600000,163.100000,143.100000,130.100000,118.800000,17.000000
3,AGO,IC.REG.DURS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.000000,83.000000,83.000000,83.000000,83.000000,68.000000,68.000000,66.000000,66.000000,66.000000,66.000000,66.000000,36.000000
4,AGO,IC.REG.PROC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.000000,12.000000,12.000000,12.000000,12.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000


In [16]:
# Selecionando o intervalo de tempo em anos
years = []

# The range goes from 1991 to 2015
for year in range(1991, 2016, 1):
    year = "YR" + str(year)
    years.append(year)
    
# Transform from wide to long format
wb_variables = pd.melt(wb_variables, id_vars=['economy', 'series'], value_vars=years, var_name='year', value_name='value')
wb_variables = wb_variables.pivot(index=['economy', 'year'], columns='series', values='value').reset_index()

In [17]:
# Removing 'YR' in each row of the year's column
wb_variables['year'] = [x.replace('YR', '') for x in wb_variables['year']]

In [18]:
# Renaming the columns
wb_variables = wb_variables.rename(columns={'NY.GDP.DEFL.KD.ZG':'inflation',
                                        'SL.UEM.TOTL.NE.ZS':'unemployment',
                                        'NE.TRD.GNFS.ZS':'exchange',
                                        'BX.KLT.DINV.WD.GD.ZS':'fdi',
                                        'NE.CON.GOVT.ZS':'governement_spending',
                                        'IC.REG.PROC':'business_procedure',
                                        'IC.REG.COST.PC.ZS':'cost_procedures',
                                        'IC.REG.DURS':'business_time',
                                        'IC.PRP.DURS':'property_time',
                                        'IC.TAX.DURS': 'tribute_time',
                                        'NY.GDP.PCAP.CD':'gdp_pc'
                                         })

In [19]:
# Missing data by column
wb_variables.isnull().mean().sort_values(ascending=False)*100

series
tribute_time            59.057325
property_time           56.127389
cost_procedures         52.789809
business_time           52.789809
business_procedure      52.789809
unemployment            38.828025
governement_spending    11.159236
exchange                 8.840764
fdi                      2.471338
inflation                2.394904
gdp_pc                   1.324841
economy                  0.000000
year                     0.000000
dtype: float64

In [20]:
# Saving countries name
countries_name = wb.economy.DataFrame(countries).reset_index()

# Only id and name columns
countries_name = countries_name[['id', 'name']]

# Renaming the id columns
countries_name = countries_name.rename(columns={'id': 'economy'})


In [21]:
# Merging countries_name and wb_variables
wb_variables = pd.merge(wb_variables, countries_name, on='economy')

In [22]:
wb_variables.head()

,economy,year,fdi,property_time,cost_procedures,business_time,business_procedure,tribute_time,governement_spending,exchange,inflation,gdp_pc,unemployment,name
0,AGO,1991,6.388693,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106.309982,850.556180,NaN,Angola
1,AGO,1992,3.466081,NaN,NaN,NaN,NaN,NaN,NaN,NaN,476.515751,657.654640,NaN,Angola
2,AGO,1993,4.964921,NaN,NaN,NaN,NaN,NaN,NaN,NaN,917.783468,466.679163,NaN,Angola
3,AGO,1994,3.837037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2175.978955,329.691784,NaN,Angola
4,AGO,1995,8.529489,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1825.495149,398.120223,NaN,Angola


In [23]:
# Just changing the columns' order
new_order = ['name', 'economy', 'year', 'fdi', 'property_time', 'cost_procedures', 'business_time', 'business_procedure', 
             'tribute_time', 'governement_spending', 'exchange', 'inflation', 'gdp_pc', 'unemployment']

wb_variables = wb_variables[new_order]

In [24]:
wb_variables.head()

,name,economy,year,fdi,property_time,cost_procedures,business_time,business_procedure,tribute_time,governement_spending,exchange,inflation,gdp_pc,unemployment
0,Angola,AGO,1991,6.388693,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106.309982,850.556180,NaN
1,Angola,AGO,1992,3.466081,NaN,NaN,NaN,NaN,NaN,NaN,NaN,476.515751,657.654640,NaN
2,Angola,AGO,1993,4.964921,NaN,NaN,NaN,NaN,NaN,NaN,NaN,917.783468,466.679163,NaN
3,Angola,AGO,1994,3.837037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2175.978955,329.691784,NaN
4,Angola,AGO,1995,8.529489,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1825.495149,398.120223,NaN


### Democracy data

In [25]:
# Importing democracy data
democracy = pd.read_excel("democracy.xlsx", usecols=['country', 'year', 'democ'])
democracy.head()

,country,year,democ
0,Afghanistan,1800,1
1,Afghanistan,1801,1
2,Afghanistan,1802,1
3,Afghanistan,1803,1
4,Afghanistan,1804,1


In [26]:
# Selecting the 1991 to 2015 year range
democracy = democracy[(democracy['year'] >= 1991) & (democracy['year'] <= 2015)]
democracy.head()

,country,year,democ
191,Afghanistan,1991,0
192,Afghanistan,1992,-77
193,Afghanistan,1993,-77
194,Afghanistan,1994,-77
195,Afghanistan,1995,-77


In [27]:
# Verifying which countries are in wb_variables and democracy 
list1 = wb_variables['name'].unique()
list2 = democracy['country'].unique()

set1 = set(list1)
set2 = set(list2)

equal_elements = set1.intersection(set2)
different_elements = set1.symmetric_difference(set2)

different_elements

{'Afghanistan',
 'Bahamas, The',
 'Belize',
 'Bosnia',
 'Bosnia and Herzegovina',
 'Brunei Darussalam',
 'Cabo Verde',
 'Cape Verde',
 'Congo Brazzaville',
 'Congo Kinshasa',
 'Congo, Dem. Rep.',
 'Congo, Rep.',
 'Congo-Brazzaville',
 "Cote D'Ivoire",
 "Cote d'Ivoire",
 'Cuba',
 'Czech Republic',
 'Czechia',
 'Czechoslovakia',
 'Djibouti',
 'Egypt',
 'Egypt, Arab Rep.',
 'Eswatini',
 'Gambia',
 'Gambia, The',
 'Hong Kong SAR, China',
 'Iceland',
 'Iran',
 'Iran, Islamic Rep.',
 'Iraq',
 'Ivory Coast',
 'Korea North',
 'Korea South',
 'Korea, Rep.',
 'Kosovo',
 'Kyrgyz Republic',
 'Kyrgyzstan',
 'Lao PDR',
 'Laos',
 'Macedonia',
 'Maldives',
 'Malta',
 'Montenegro',
 'Myanmar',
 'Myanmar (Burma)',
 'Panama',
 'Russia',
 'Russian Federation',
 'Serbia',
 'Serbia and Montenegro',
 'Somalia',
 'South Sudan',
 'Sudan',
 'Sudan-North',
 'Swaziland',
 'Syria',
 'Syrian Arab Republic',
 'Taiwan',
 'Timor Leste',
 'Turkey',
 'Turkiye',
 'Turkmenistan',
 'UAE',
 'USSR',
 'United Arab Emirates',


Some of the countries shown above has different names depending on the data source, bellow these countries will be renamed to have the same name

In [28]:
democracy['country'] = [x.replace('Ivory Coast', "Cote d'Ivoire") for x in democracy['country']]
democracy['country'] = [x.replace("Cote D'Ivoire", "Cote d'Ivoire") for x in democracy['country']]
democracy['country'] = [x.replace("Congo-Brazzaville", "Congo Brazzaville") for x in democracy['country']]
democracy['country'] = [x.replace("Congo Brazzaville", "Congo, Rep.") for x in democracy['country']]
democracy['country'] = [x.replace("Congo Kinshasa", "Congo, Dem. Rep.") for x in democracy['country']]
democracy['country'] = [x.replace("Korea South", "Korea, Rep.") for x in democracy['country']]
democracy['country'] = [x.replace("Cape Verde", "Cabo Verde") for x in democracy['country']]
democracy['country'] = [x.replace("Czech Republic", "Czechia") for x in democracy['country']]
democracy['country'] = [x.replace("Egypt", "Egypt, Arab Rep.") for x in democracy['country']]
democracy['country'] = [x.replace("Swaziland", "Eswatini") for x in democracy['country']]
democracy['country'] = [x.replace("Gambia", "Gambia, The") for x in democracy['country']]
democracy['country'] = [x.replace("Iran", "Iran, Islamic Rep.") for x in democracy['country']]
democracy['country'] = [x.replace("Kyrgyzstan", "Kyrgyz Republic") for x in democracy['country']]
democracy['country'] = [x.replace("Laos", "Lao PDR") for x in democracy['country']]
democracy['country'] = [x.replace("Myanmar (Burma)", "Myanmar") for x in democracy['country']]
democracy['country'] = [x.replace("Russia", "Russian Federation") for x in democracy['country']]
democracy['country'] = [x.replace("Syria", "Syrian Arab Republic") for x in democracy['country']]
democracy['country'] = [x.replace("Turkey", "Turkiye") for x in democracy['country']]
democracy['country'] = [x.replace("UAE", "United Arab Emirates") for x in democracy['country']]
democracy['country'] = [x.replace("Venezuela", "Venezuela, RB") for x in democracy['country']]
democracy['country'] = [x.replace("Yemen", "Yemen, Rep.") for x in democracy['country']]
democracy['country'] = [x.replace("Egypt, Arab Rep.", "Egypt, Arab Rep.") for x in democracy['country']]

In [29]:
# Removing countries that are in democracy but not in variables_wb and removing other countries in specific 
countries_to_drop = ['Korea North', 'Afghanistan', 'Bosnia', 'Cuba', 'Czechoslovakia','Djibouti', 'Iraq', 'Kosovo', 'Macedonia',
                    'Maldives', 'Montenegro', 'Panama', 'Serbia and Montenegro', 'Yugoslavia', 'Somalia',
                     'South Sudan', 'Sudan', 'Sudan-North', 'Taiwan', 'Timor Leste', 'USSR', 'Uzbekistan',
                    'Serbia', 'Turkmenistan']

democracy = democracy[~democracy['country'].isin(countries_to_drop)]

In [30]:
# removing countries that are in wb_variables but not in democracy
countries_to_drop2 = ['Bahamas, The', 'Belize', 'Bosnia and Herzegovina', 'Brunei Darussalam', 'Hong Kong SAR, China',
                      'Iceland', 'Maldives', 'Malta']

wb_variables = wb_variables[~wb_variables['name'].isin(countries_to_drop2)]

In [31]:
# Renaming the column country in democracy for a merging with the wb_variables data frame
democracy = democracy.rename(columns={'country':'name'})

# Object to int
wb_variables['year'] = wb_variables['year'].astype('int64')

# Merging wb_variables and democracy
df1 = pd.merge(wb_variables, democracy, left_on=['name', 'year'], right_on=['name', 'year'])

In [32]:
df1.head()

,name,economy,year,fdi,property_time,cost_procedures,business_time,business_procedure,tribute_time,governement_spending,exchange,inflation,gdp_pc,unemployment,democ
0,Angola,AGO,1991,6.388693,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106.309982,850.556180,NaN,-88
1,Angola,AGO,1992,3.466081,NaN,NaN,NaN,NaN,NaN,NaN,NaN,476.515751,657.654640,NaN,-77
2,Angola,AGO,1993,4.964921,NaN,NaN,NaN,NaN,NaN,NaN,NaN,917.783468,466.679163,NaN,-88
3,Angola,AGO,1994,3.837037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2175.978955,329.691784,NaN,-88
4,Angola,AGO,1995,8.529489,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1825.495149,398.120223,NaN,-88


### Tax burden

In [36]:
# Importing the tax burden data
tax_burden = pd.read_csv('tax_burden.csv', usecols=['Name', 'Index Year', 'Tax Burden'])

# Sorting the data
tax_burden = tax_burden.sort_values(['Name', 'Index Year'], ascending=[True, True])

# Renaming the columns
tax_burden = tax_burden.rename(columns={'Name':'name', 
                         'Index Year': 'year',
                         'Tax burden': 'carga_trib'})

# Selecting the range between 1995 and 2015
tax_burden = tax_burden[(tax_burden['year'] >= 1995) & (tax_burden['year'] <= 2015)]

In [39]:
# Checking which different elements (countries) are in the df1 and tax_burden
list1 = df1['name'].unique()
list2 = tax_burden['name'].unique()

set1 = set(list1)
set2 = set(list2)

equal_elements = set1.intersection(set2)
different_elements = set1.symmetric_difference(set2)

different_elements

{'Afghanistan',
 'Bangladesh',
 'Bangladesh ',
 'Barbados',
 'Belize',
 'Bosnia and Herzegovina',
 'Brunei Darussalam',
 'Burma',
 'Cape Verde',
 'Congo, Dem. Rep.',
 'Congo, Rep.',
 'Costa Rica',
 'Costa Rica ',
 "Cote d'Ivoire",
 'Cuba',
 'Czech Republic',
 'Czechia',
 "Côte d'Ivoire ",
 'Democratic Republic of Congo',
 'Djibouti',
 'Dominica',
 'Egypt',
 'Egypt, Arab Rep.',
 'El Salvador',
 'El Salvador ',
 'Eswatini',
 'Gambia, The',
 'Guatemala',
 'Guatemala ',
 'Honduras',
 'Honduras ',
 'Hong Kong',
 'Hungary',
 'Hungary ',
 'Iceland',
 'Iran',
 'Iran, Islamic Rep.',
 'Iraq',
 'Jamaica',
 'Jamaica ',
 'Kiribati',
 'Korea, Rep.',
 'Kosovo',
 'Kyrgyz Republic',
 'Kyrgyz Republic ',
 'Lao PDR',
 'Laos',
 'Liechtenstein',
 'Macau',
 'Macedonia',
 'Malaysia',
 'Malaysia ',
 'Maldives',
 'Malta',
 'Micronesia',
 'Montenegro',
 'Mozambique',
 'Mozambique ',
 'Myanmar',
 'Netherlands',
 'Nicaragua',
 'Nicaragua ',
 'North Korea',
 'Pakistan',
 'Pakistan ',
 'Panama ',
 'Paraguay',
 'Par

Repeting the same process made with the democracy data


In [40]:
# Removing the blank space at the end of the countries' name in each row
tax_burden['name'] = tax_burden['name'].str.rstrip()

In [41]:
tax_burden['name'] = [x.replace('Cape Verde', "Cabo Verde") for x in tax_burden['name']]
tax_burden['name'] = [x.replace('Burma', "Myanmar") for x in tax_burden['name']]
tax_burden['name'] = [x.replace('Turkey', "Turkiye") for x in tax_burden['name']]
tax_burden['name'] = [x.replace('The Netherlands', "Netherlands") for x in tax_burden['name']]
tax_burden['name'] = [x.replace('Democratic Republic of Congo', "Congo, Dem. Rep.") for x in tax_burden['name']]
tax_burden['name'] = [x.replace('Republic of Congo', "Congo, Rep.") for x in tax_burden['name']]
tax_burden['name'] = [x.replace("Côte d'Ivoire", "Cote d'Ivoire") for x in tax_burden['name']]
tax_burden['name'] = [x.replace('Czech Republic', "Czechia") for x in tax_burden['name']]
tax_burden['name'] = [x.replace('Egypt', "Egypt, Arab Rep.") for x in tax_burden['name']]
tax_burden['name'] = [x.replace('Swaziland', "Eswatini") for x in tax_burden['name']]
tax_burden['name'] = [x.replace('The Gambia', "Gambia, The") for x in tax_burden['name']]
tax_burden['name'] = [x.replace('Iran', "Iran, Islamic Rep.") for x in tax_burden['name']]
tax_burden['name'] = [x.replace('South Korea', "Korea, Rep.") for x in tax_burden['name']]
tax_burden['name'] = [x.replace('Laos', "Lao PDR") for x in tax_burden['name']]
tax_burden['name'] = [x.replace('The Philippines', "Philippines") for x in tax_burden['name']]
tax_burden['name'] = [x.replace('Russia', "Russian Federation") for x in tax_burden['name']]
tax_burden['name'] = [x.replace('Slovakia', "Slovak Republic") for x in tax_burden['name']]
tax_burden['name'] = [x.replace('Syria', "Syrian Arab Republic") for x in tax_burden['name']]
tax_burden['name'] = [x.replace('Burma', "Myanmar") for x in tax_burden['name']]
tax_burden['name'] = [x.replace("Venezuela", "Venezuela, RB") for x in tax_burden['name']] 
tax_burden['name'] = [x.replace("Yemen", "Yemen, Rep.") for x in tax_burden['name']]

In [42]:
# Countries that are in tax_burden but not in df1
countries_to_drop3 = ['Afghanistan', 'Barbados', 'Belize', 'Bosnia and Herzegovina', 'Brunei Darussalam', 'Cuba', 'Djibouti', 
                      'Dominica', 'Hong Kong', 'Iceland', 'Iraq', 'Kiribati', 'Kosovo', 'Liechtenstein', 'Macau', 'Macedonia', 
                      'Maldives', 'Malta', 'Micronesia', 'Montenegro', 'North Korea', 'Panama', 'Saint Lucia', 
                      'Saint Vincent and the Grenadines', 'Samoa', 'Serbia', 'Seychelles', 'Somalia', 'Sudan', 
                      'São Tomé and Príncipe', 'Taiwan', 'The Bahamas', 'Timor-Leste', 'Tonga', 'Turkmenistan', 
                      'Uzbekistan', 'Vanuatu']

tax_burden = tax_burden[~tax_burden['name'].isin(countries_to_drop3)]

In [43]:
# Merging df1 and tax_burden
df2 = pd.merge(df1, tax_burden, left_on=['name', 'year'], right_on=['name', 'year'])
df2 = df2.rename(columns={'Tax Burden': 'tax_burden'})

In [44]:
df2.head()

,name,economy,year,fdi,property_time,cost_procedures,business_time,business_procedure,tribute_time,governement_spending,exchange,inflation,gdp_pc,unemployment,democ,tax_burden
0,Angola,AGO,1995,8.529489,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1825.495149,398.120223,NaN,-88,61.6
1,Angola,AGO,1996,2.762770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4800.531644,454.375004,NaN,-88,54.6
2,Angola,AGO,1997,5.363360,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.453022,516.127849,NaN,0,52.6
3,Angola,AGO,1998,17.121191,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.359348,423.403332,NaN,0,59.1
4,Angola,AGO,1999,40.167251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,557.501113,387.689415,NaN,0,47.9
